https://archive.ics.uci.edu/ml/datasets/Online+News+Popularity

In [33]:
import pandas as pd
import numpy as np

df = pd.read_csv('OnlineNewsPopularity.csv')


In [34]:
df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505


In [35]:
data.describe()

,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,...,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity
count,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,...,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000
mean,10.398749,546.514731,0.548216,0.996469,0.689175,10.883690,3.293638,4.544143,1.249874,4.548239,...,0.353825,0.095446,0.756728,-0.259524,-0.521944,-0.107500,0.282353,0.071425,0.341843,0.156064
std,2.114037,471.107508,3.520708,5.231231,3.264816,11.332017,3.855141,8.309434,4.107855,0.844406,...,0.104542,0.071315,0.247786,0.127726,0.290290,0.095373,0.324247,0.265450,0.188791,0.226294
min,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000
25%,9.000000,246.000000,0.470870,1.000000,0.625739,4.000000,1.000000,1.000000,0.000000,4.478404,...,0.306244,0.050000,0.600000,-0.328383,-0.700000,-0.125000,0.000000,0.000000,0.166667,0.000000
50%,10.000000,409.000000,0.539226,1.000000,0.690476,8.000000,3.000000,1.000000,0.000000,4.664082,...,0.358755,0.100000,0.800000,-0.253333,-0.500000,-0.100000,0.150000,0.000000,0.500000,0.000000
75%,12.000000,716.000000,0.608696,1.000000,0.754630,14.000000,4.000000,4.000000,1.000000,4.854839,...,0.411428,0.100000,1.000000,-0.186905,-0.300000,-0.050000,0.500000,0.150000,0.500000,0.250000
max,23.000000,8474.000000,701.000000,1042.000000,650.000000,304.000000,116.000000,128.000000,91.000000,8.041534,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.500000,1.000000


Odrzucanie url i timedelta jako nierelewantne

In [36]:
data = df.iloc[:,2:-1]
target = df.iloc[:,-1]

In [37]:
data.head()

,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,...,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity
0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,0.0,4.680365,...,0.378636,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500
1,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,0.0,4.913725,...,0.286915,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000
2,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,0.0,4.393365,...,0.495833,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000
3,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,0.0,4.404896,...,0.385965,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000
4,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,0.0,4.682836,...,0.411127,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364


%matplotlib inline

import matplotlib.pyplot as plt

data.hist(bins=100, figsize=(100,100))
plt.show()

import seaborn as sb

sb.pairplot(data, diag_kind="kde")

from pandas.plotting import scatter_matrix

scatter_m = scatter_matrix(data, figsize=(100,100), diagonal="kde")

In [38]:
data_np = np.array(data, dtype=np.int16)
target_np = np.array(target, dtype=np.int16)

In [39]:
print("First client in database")
print(data_np[1,:])
print('---Mean---')
print(data_np.mean(axis=0))
print('--std---')
print(data_np.std(axis=0))

First client in database
[  9 255   0   0   0   3   1   1   0   4   4   0   0   1   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0]
---Mean---
[ 1.03987489e+01  5.46514731e+02  1.76823731e-02  2.62839269e-02
  1.63959237e-02  1.08836898e+01  3.29363838e+00  4.54414287e+00
  1.24987388e+00  4.00110988e+00  7.22376652e+00  5.29462214e-02
  1.78009283e-01  1.57854909e-01  5.85965089e-02  1.85299163e-01
  2.12566845e-01  2.61068005e+01  1.02335413e+03  3.06996595e+02
  2.93159293e+03 -8.08959247e+03  5.41453511e+02  1.11691212e+03
  5.20214832e+03  3.12544166e+03  2.27125247e+03  4.14326488e+03
  3.46237347e+03  1.68020381e-01  1.86409040e-01  1.87544143e-01
  1.83306427e-01  1.43804863e-01  6.18756937e-02  6.90394511e-02
  1.30915145e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  1.00897992e-04  0.00000000e+00
  0.00000000

In [40]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_data = scaler.fit_transform(data_np)

In [41]:
print("First client in database")
print(scaled_data[1,:])
print('---Mean---')
print(scaled_data.mean(axis=0))
print('--std---')
print(scaled_data.std(axis=0))

First client in database
[-6.61656647e-01 -6.18793806e-01 -5.02246297e-03 -5.02246297e-03
 -5.02246297e-03 -6.95709247e-01 -5.94963251e-01 -4.26525775e-01
 -3.04268192e-01 -1.42550724e-03 -1.68862599e+00 -2.36445017e-01
 -4.65358733e-01  2.30974702e+00 -2.49487028e-01 -4.76911337e-01
 -5.19566229e-01 -3.74923515e-01 -4.98382469e-01 -5.38986774e-01
 -3.26119282e-01  6.66611322e-01 -2.84326065e-02 -9.82123919e-01
 -1.46622155e+00 -2.38479474e+00 -4.57481217e-01 -5.10458697e-01
 -5.38164453e-01  2.22523233e+00 -4.78663628e-01 -4.80454058e-01
 -4.73760968e-01 -4.09826850e-01 -2.56820594e-01 -2.72322187e-01
 -3.88118062e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00453060e-02  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.88764891e-01 -2.79744773e-02
 -1.42069249e-02 -1.42069249e-02 -7.72701259e-01  3.58902004e-02
  4.03593317e-01  3.58902004e-02 -2.60925298e-01 -7.02586927e-02
  0.00000000e+00 -1.36581870e-01]
---Mean---
[ 3.78535531e-16  6.

In [49]:
from sklearn.model_selection import train_test_split

news_data, news_data, \
news_target, news_target = \
train_test_split(scaled_data,target, test_size=0.2, random_state=10)

In [56]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

linear_reg = LinearRegression()
scores = cross_val_score(linear_reg, scaled_data, target, scoring='r2', cv=5)
print(scores)

[-1.13989748e-02  1.47866253e-02  8.75613921e-03 -2.92651124e+25
  1.92314219e-02]


In [58]:
print("Wyniki: ", scores)
print("Średnia: ", scores.mean())    
print("Odchylenie standardowe: ", scores.std())    

Wyniki:  [-1.13989748e-02  1.47866253e-02  8.75613921e-03 -2.92651124e+25
  1.92314219e-02]
Średnia:  -5.853022489791515e+24
Odchylenie standardowe:  1.170604497958303e+25


In [59]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDRegressor


scores = cross_val_score(SGDRegressor(), scaled_data, target, scoring='r2', cv=5)
print(scores)

[-8.05909996e+12 -1.40269631e+12 -1.51857862e+13 -9.91943720e+13
 -2.09160956e+13]


In [60]:
print("Wyniki: ", scores)
print("Średnia: ", scores.mean())    
print("Odchylenie standardowe: ", scores.std())    

Wyniki:  [-8.05909996e+12 -1.40269631e+12 -1.51857862e+13 -9.91943720e+13
 -2.09160956e+13]
Średnia:  -28951610017277.43
Odchylenie standardowe:  35731086292764.69


In [67]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor

scores = cross_val_score(KNeighborsRegressor(), scaled_data, target, scoring='r2', cv=5)
print(scores)

[-0.09121758 -0.09629593 -0.06135123 -0.31152547 -0.13001171]


In [68]:
print("Wyniki: ", scores)
print("Średnia: ", scores.mean())    
print("Odchylenie standardowe: ", scores.std())    

Wyniki:  [-0.09121758 -0.09629593 -0.06135123 -0.31152547 -0.13001171]
Średnia:  -0.13808038287380048
Odchylenie standardowe:  0.08941781547717793


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

param_grid = {
    'n_neighbors': [3,5,7,10], 'weights':['uniform','distance'], 'leaf_size':[30,45,60,90]
}


rnd_search = GridSearchCV(KNeighborsRegressor(), param_grid, scoring="r2", cv=5, return_train_score=True, verbose=3)
rnd_search.fit(scaled_data, target)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV 1/5] END leaf_size=30, n_neighbors=3, weights=uniform;, score=(train=0.392, test=-0.192) total time=   0.6s
[CV 2/5] END leaf_size=30, n_neighbors=3, weights=uniform;, score=(train=0.439, test=-0.145) total time=   0.6s
[CV 3/5] END leaf_size=30, n_neighbors=3, weights=uniform;, score=(train=0.382, test=-0.105) total time=   0.6s
[CV 4/5] END leaf_size=30, n_neighbors=3, weights=uniform;, score=(train=0.410, test=-0.685) total time=   0.6s
